In [1]:
import pandas as pd
import haversine as hav
from haversine import Unit
import numpy as np

In [2]:
EADA = pd.read_csv('EADA_2019_filtered_coord.csv')
mlb_geo = pd.read_csv('dataMLB.csv')
nba_geo = pd.read_csv('dataNBA.csv')

In [3]:
EADA.head()

,unitid,institution_name,addr1_txt,addr2_txt,city_txt,state_cd,zip_text,ClassificationCode,classification_name,ClassificationOther,...,GRND_TOTAL_REVENUE,GRND_TOTAL_EXPENSE,OPEXPPERTEAM_MEN_Baseball,OPEXPPERTEAM_MEN_Bskball,REV_MEN_Baseball,REV_MEN_Bskball,EXP_MEN_Baseball,EXP_MEN_Bskball,lat,lon
0,100654,Alabama A & M University,4900 Meridian Street,NaN,Normal,AL,35762.0,2,NCAA Division I-FCS,NaN,...,13790500,13626724,171178.0,184268.0,608854.0,1591652.0,588354.0,1667122.0,34.782600,-86.568700
1,100706,University of Alabama in Huntsville,301 Sparkman Dr,NaN,Huntsville,AL,35899.0,5,NCAA Division II without football,NaN,...,7781027,7781027,173401.0,143803.0,541784.0,705934.0,541784.0,705934.0,34.725000,-86.640000
2,100724,Alabama State University,915 S Jackson Street,NaN,Montgomery,AL,36104.0,2,NCAA Division I-FCS,NaN,...,11639722,11639722,72110.0,180911.0,628125.0,992694.0,628125.0,992694.0,32.364000,-86.295000
3,100751,The University of Alabama,739 University Blvd,NaN,Tuscaloosa,AL,35487.0,1,NCAA Division I-FBS,NaN,...,166812799,166812799,1166447.0,1457096.0,2016412.0,16028795.0,4173211.0,15966875.0,33.210833,-87.546111
4,100760,Central Alabama Community College,1675 Cherokee Rd,NaN,Alexander City,AL,35010.0,12,NJCAA Division I,NaN,...,906077,889789,61904.0,NaN,318500.0,NaN,308353.0,NaN,32.926230,-85.946490


In [4]:
school_geo = EADA[['unitid', 'institution_name', 'state_cd', 'lat', 'lon']]

In [5]:
school_geo.isna().sum()

unitid              0
institution_name    0
state_cd            1
lat                 0
lon                 0
dtype: int64

In [6]:
region_dict = {'AL':'ESC', 
'AK': 'oth', 
'AZ': 'MNT',
'AR': 'WSC', 
'CA': 'PAC',
'CO': 'MNT',
'CT': 'NE',
'DC': 'SA',
'DE': 'SA',
'FL': 'SA',
'GA': 'SA',
'HI': 'oth',
'ID': 'MNT',
'IL': 'ENC',
'IN': 'ENC',
'IA': 'WNC',
'KS': 'WNC',
'KY': 'ESC',
'LA': 'WSC',
'ME': 'NE',
'MD': 'SA',
'MA': 'NE',
'MI': 'ENC',
'MN': 'WNC',
'MS': 'ESC',
'MO': 'WNC',
'MT': 'MNT',
'NE': 'WNC',
'NV': 'MNT',
'NH': 'NE',
'NJ': 'MA',
'NM': 'MNT',
'NY': 'MA',
'NC': 'SA',
'ND': 'WNC',
'OH': 'ENC',
'OK': 'WSC',
'OR': 'PAC',
'PA': 'MA',
'RI': 'NE',
'SC': 'SA',
'SD': 'WNC',
'TN': 'ESC',
'TX': 'WSC',
'UT': 'MNT',
'VT': 'NE',
'VA': 'SA',
'WA': 'PAC',
'WV': 'SA',
'WI': 'ENC',
'WY': 'MNT',
'AS': 'oth',
'GU': 'oth',
'MH': 'oth',
'FM': 'oth',
'MP': 'oth',
'PW': 'oth',
'PR': 'oth',
'VI': 'oth'}

In [7]:
region_list = []

for index, row in school_geo.iterrows():
    try:
        reg = region_dict[row["state_cd"]]
        region_list.append(reg)
        
    except:
        school_geo = school_geo.drop(index = index)

region_series = pd.Series(region_list)
school_geo['region'] = region_series

In [8]:
school_geo['nearest_mlb'] = ''
school_geo['nearest_mlb_dist'] = np.nan

In [9]:
for index, row in school_geo.iterrows():
    nearest_mlb = ''
    nearest_mlb_dist = 1000000000
    
    school_coord = (row['lat'], row['lon'])
   
    for index2, team in mlb_geo.iterrows():
        mlb_coord = (team['Latitude'], team['Longitude'])
        dist = hav.haversine(school_coord, mlb_coord, unit = Unit.MILES)
        
        if dist < nearest_mlb_dist:
            nearest_mlb = team['Team']
            nearest_mlb_dist = dist
            
            school_geo.at[index, 'nearest_mlb'] = nearest_mlb
            school_geo.at[index, 'nearest_mlb_dist'] = nearest_mlb_dist
            
    print(f'Row {index} done.')

Row 0 done.
Row 1 done.
Row 2 done.
Row 3 done.
Row 4 done.
Row 5 done.
Row 6 done.
Row 7 done.
Row 8 done.
Row 9 done.
Row 10 done.
Row 11 done.
Row 12 done.
Row 13 done.
Row 14 done.
Row 15 done.
Row 16 done.
Row 17 done.
Row 18 done.
Row 19 done.
Row 20 done.
Row 21 done.
Row 22 done.
Row 23 done.
Row 24 done.
Row 25 done.
Row 26 done.
Row 27 done.
Row 28 done.
Row 29 done.
Row 30 done.
Row 31 done.
Row 32 done.
Row 33 done.
Row 34 done.
Row 35 done.
Row 36 done.
Row 37 done.
Row 38 done.
Row 39 done.
Row 40 done.
Row 41 done.
Row 42 done.
Row 43 done.
Row 44 done.
Row 45 done.
Row 46 done.
Row 47 done.
Row 48 done.
Row 49 done.
Row 50 done.
Row 51 done.
Row 52 done.
Row 53 done.
Row 54 done.
Row 55 done.
Row 56 done.
Row 57 done.
Row 58 done.
Row 59 done.
Row 60 done.
Row 61 done.
Row 62 done.
Row 63 done.
Row 64 done.
Row 65 done.
Row 66 done.
Row 67 done.
Row 68 done.
Row 69 done.
Row 70 done.
Row 71 done.
Row 72 done.
Row 73 done.
Row 74 done.
Row 75 done.
Row 76 done.
Row 77 do

Row 637 done.
Row 638 done.
Row 639 done.
Row 640 done.
Row 641 done.
Row 642 done.
Row 643 done.
Row 644 done.
Row 645 done.
Row 646 done.
Row 647 done.
Row 648 done.
Row 649 done.
Row 650 done.
Row 651 done.
Row 652 done.
Row 653 done.
Row 654 done.
Row 655 done.
Row 656 done.
Row 657 done.
Row 658 done.
Row 659 done.
Row 660 done.
Row 661 done.
Row 662 done.
Row 663 done.
Row 664 done.
Row 665 done.
Row 666 done.
Row 667 done.
Row 668 done.
Row 669 done.
Row 670 done.
Row 671 done.
Row 672 done.
Row 673 done.
Row 674 done.
Row 675 done.
Row 676 done.
Row 677 done.
Row 678 done.
Row 679 done.
Row 680 done.
Row 681 done.
Row 682 done.
Row 683 done.
Row 684 done.
Row 685 done.
Row 686 done.
Row 687 done.
Row 688 done.
Row 689 done.
Row 690 done.
Row 691 done.
Row 692 done.
Row 693 done.
Row 694 done.
Row 695 done.
Row 696 done.
Row 697 done.
Row 698 done.
Row 699 done.
Row 700 done.
Row 701 done.
Row 702 done.
Row 703 done.
Row 704 done.
Row 705 done.
Row 706 done.
Row 707 done.
Row 70

Row 1247 done.
Row 1248 done.
Row 1249 done.
Row 1250 done.
Row 1251 done.
Row 1252 done.
Row 1253 done.
Row 1254 done.
Row 1255 done.
Row 1256 done.
Row 1257 done.
Row 1258 done.
Row 1259 done.
Row 1260 done.
Row 1261 done.
Row 1262 done.
Row 1263 done.
Row 1264 done.
Row 1265 done.
Row 1266 done.
Row 1267 done.
Row 1268 done.
Row 1269 done.
Row 1270 done.
Row 1271 done.
Row 1272 done.
Row 1273 done.
Row 1274 done.
Row 1275 done.
Row 1276 done.
Row 1277 done.
Row 1278 done.
Row 1279 done.
Row 1280 done.
Row 1281 done.
Row 1282 done.
Row 1283 done.
Row 1284 done.
Row 1285 done.
Row 1286 done.
Row 1287 done.
Row 1288 done.
Row 1289 done.
Row 1290 done.
Row 1291 done.
Row 1292 done.
Row 1293 done.
Row 1294 done.
Row 1295 done.
Row 1296 done.
Row 1297 done.
Row 1298 done.
Row 1299 done.
Row 1300 done.
Row 1301 done.
Row 1302 done.
Row 1303 done.
Row 1304 done.
Row 1305 done.
Row 1306 done.
Row 1307 done.
Row 1308 done.
Row 1309 done.
Row 1310 done.
Row 1311 done.
Row 1312 done.
Row 1313 d

In [10]:
school_geo['nearest_nba'] = ''
school_geo['nearest_nba_dist'] = np.nan

In [11]:
for index, row in school_geo.iterrows():
    nearest_nba = ''
    nearest_nba_dist = 1000000000
    
    school_coord = (row['lat'], row['lon'])
   
    for index2, team in nba_geo.iterrows():
        nba_coord = (team['Latitude'], team['Longitude'])
        dist = hav.haversine(school_coord, nba_coord, unit = Unit.MILES)
        
        if dist < nearest_nba_dist:
            nearest_nba = team['Team']
            nearest_nba_dist = dist
            
            school_geo.at[index, 'nearest_nba'] = nearest_nba
            school_geo.at[index, 'nearest_nba_dist'] = nearest_nba_dist
            
    print(f'Row {index} done.')

Row 0 done.
Row 1 done.
Row 2 done.
Row 3 done.
Row 4 done.
Row 5 done.
Row 6 done.
Row 7 done.
Row 8 done.
Row 9 done.
Row 10 done.
Row 11 done.
Row 12 done.
Row 13 done.
Row 14 done.
Row 15 done.
Row 16 done.
Row 17 done.
Row 18 done.
Row 19 done.
Row 20 done.
Row 21 done.
Row 22 done.
Row 23 done.
Row 24 done.
Row 25 done.
Row 26 done.
Row 27 done.
Row 28 done.
Row 29 done.
Row 30 done.
Row 31 done.
Row 32 done.
Row 33 done.
Row 34 done.
Row 35 done.
Row 36 done.
Row 37 done.
Row 38 done.
Row 39 done.
Row 40 done.
Row 41 done.
Row 42 done.
Row 43 done.
Row 44 done.
Row 45 done.
Row 46 done.
Row 47 done.
Row 48 done.
Row 49 done.
Row 50 done.
Row 51 done.
Row 52 done.
Row 53 done.
Row 54 done.
Row 55 done.
Row 56 done.
Row 57 done.
Row 58 done.
Row 59 done.
Row 60 done.
Row 61 done.
Row 62 done.
Row 63 done.
Row 64 done.
Row 65 done.
Row 66 done.
Row 67 done.
Row 68 done.
Row 69 done.
Row 70 done.
Row 71 done.
Row 72 done.
Row 73 done.
Row 74 done.
Row 75 done.
Row 76 done.
Row 77 do

Row 620 done.
Row 621 done.
Row 622 done.
Row 623 done.
Row 624 done.
Row 625 done.
Row 626 done.
Row 627 done.
Row 628 done.
Row 629 done.
Row 630 done.
Row 631 done.
Row 632 done.
Row 633 done.
Row 634 done.
Row 635 done.
Row 636 done.
Row 637 done.
Row 638 done.
Row 639 done.
Row 640 done.
Row 641 done.
Row 642 done.
Row 643 done.
Row 644 done.
Row 645 done.
Row 646 done.
Row 647 done.
Row 648 done.
Row 649 done.
Row 650 done.
Row 651 done.
Row 652 done.
Row 653 done.
Row 654 done.
Row 655 done.
Row 656 done.
Row 657 done.
Row 658 done.
Row 659 done.
Row 660 done.
Row 661 done.
Row 662 done.
Row 663 done.
Row 664 done.
Row 665 done.
Row 666 done.
Row 667 done.
Row 668 done.
Row 669 done.
Row 670 done.
Row 671 done.
Row 672 done.
Row 673 done.
Row 674 done.
Row 675 done.
Row 676 done.
Row 677 done.
Row 678 done.
Row 679 done.
Row 680 done.
Row 681 done.
Row 682 done.
Row 683 done.
Row 684 done.
Row 685 done.
Row 686 done.
Row 687 done.
Row 688 done.
Row 689 done.
Row 690 done.
Row 69

Row 1201 done.
Row 1202 done.
Row 1203 done.
Row 1204 done.
Row 1205 done.
Row 1206 done.
Row 1207 done.
Row 1208 done.
Row 1209 done.
Row 1210 done.
Row 1211 done.
Row 1212 done.
Row 1213 done.
Row 1214 done.
Row 1215 done.
Row 1216 done.
Row 1217 done.
Row 1218 done.
Row 1219 done.
Row 1220 done.
Row 1221 done.
Row 1222 done.
Row 1223 done.
Row 1224 done.
Row 1225 done.
Row 1226 done.
Row 1227 done.
Row 1228 done.
Row 1229 done.
Row 1230 done.
Row 1231 done.
Row 1232 done.
Row 1233 done.
Row 1234 done.
Row 1235 done.
Row 1236 done.
Row 1237 done.
Row 1238 done.
Row 1239 done.
Row 1240 done.
Row 1241 done.
Row 1242 done.
Row 1243 done.
Row 1244 done.
Row 1245 done.
Row 1246 done.
Row 1247 done.
Row 1248 done.
Row 1249 done.
Row 1250 done.
Row 1251 done.
Row 1252 done.
Row 1253 done.
Row 1254 done.
Row 1255 done.
Row 1256 done.
Row 1257 done.
Row 1258 done.
Row 1259 done.
Row 1260 done.
Row 1261 done.
Row 1262 done.
Row 1263 done.
Row 1264 done.
Row 1265 done.
Row 1266 done.
Row 1267 d

Row 1758 done.
Row 1759 done.
Row 1760 done.
Row 1761 done.
Row 1762 done.
Row 1763 done.
Row 1764 done.
Row 1765 done.
Row 1766 done.
Row 1767 done.
Row 1768 done.
Row 1769 done.
Row 1770 done.
Row 1771 done.
Row 1772 done.
Row 1773 done.
Row 1774 done.
Row 1775 done.
Row 1776 done.
Row 1777 done.
Row 1778 done.
Row 1779 done.
Row 1780 done.
Row 1781 done.
Row 1782 done.
Row 1783 done.
Row 1784 done.
Row 1785 done.
Row 1786 done.
Row 1787 done.
Row 1788 done.
Row 1789 done.
Row 1790 done.
Row 1791 done.
Row 1792 done.
Row 1793 done.
Row 1794 done.
Row 1795 done.
Row 1796 done.
Row 1797 done.
Row 1798 done.
Row 1799 done.
Row 1800 done.
Row 1801 done.
Row 1802 done.
Row 1803 done.
Row 1804 done.
Row 1805 done.
Row 1806 done.
Row 1807 done.
Row 1808 done.
Row 1809 done.
Row 1810 done.


In [12]:
school_geo.head()

,unitid,institution_name,state_cd,lat,lon,region,nearest_mlb,nearest_mlb_dist,nearest_nba,nearest_nba_dist
0,100654,Alabama A & M University,AL,34.782600,-86.568700,ESC,Atlanta Braves,134.784204,Atlanta Hawks,142.837358
1,100706,University of Alabama in Huntsville,AL,34.725000,-86.640000,ESC,Atlanta Braves,136.725585,Atlanta Hawks,144.541163
2,100724,Alabama State University,AL,32.364000,-86.295000,ESC,Atlanta Braves,149.301421,Atlanta Hawks,146.123655
3,100751,The University of Alabama,AL,33.210833,-87.546111,ESC,Atlanta Braves,183.343203,Atlanta Hawks,185.384011
4,100760,Central Alabama Community College,AL,32.926230,-85.946490,ESC,Atlanta Braves,108.192247,Atlanta Hawks,106.309279


In [13]:
school_geo.to_csv('school_geo.csv', index = False)